In [1]:
import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
import warnings
warnings.filterwarnings('ignore', 'SettingWithCopyWarning')
importlib.reload(co2)

<module 'auto_co2' from '/home/mln/GIT/car-co2-prediction_pre-release/src/auto_co2/__init__.py'>

df = pd.read_csv('data/sample.csv')
df = co2.data.dataviz_preprocessing(df)

df = pd.read_csv('data/sample.csv')
df.head()

df = co2.data.load_co2_data()
co2.styles.displayer(df)



auth_file_path = '/home/mln/.kaggle/kaggle.json'
filename = 'test.csv'  # replace with the actual name of the csv file in the zip

df = co2.data.download_and_load_co2_data(auth_file_path)
df = df.sample(1000)

df = co2.data.load_co2_data()
df = df.sample(5000)

co2.data.save_processed_data(df, classification=False, filepath='../data/sample')

In [2]:
df = pd.read_pickle('../data/sample/co2_regression_20240112-151326.pkl')
df = co2.data.rename_columns(df)
df.columns

Index(['ID', 'Country', 'VehicleFamilyIdentification', 'Pool',
       'ManufacturerName', 'ManufNameOem', 'ManufNameMS', 'TypeApprovalNumber',
       'Type', 'Variant', 'Version', 'Make', 'CommercialName',
       'VehicleCategory', 'CategoryOf', 'TotalNewRegistrations',
       'MassRunningOrder', 'WltpTestMass', 'Co2EmissionsNedc',
       'Co2EmissionsWltp', 'BaseWheel', 'AxleWidthSteering', 'AxleWidthOther',
       'FuelType', 'FuelMode', 'EngineCapacity', 'EnginePower',
       'ElectricConsumption', 'InnovativeTechnology',
       'InnovativeEmissionsReduction', 'InnovativeEmissionsReductionWltp',
       'DeviationFactor', 'VerificationFactor', 'Status', 'RegistrationYear',
       'RegistrationDate', 'FuelConsumption', 'ElectricRange'],
      dtype='object')

In [3]:
df = co2.data.ml_preprocessing(df,
                               rem_axlewidth=True,
                               rem_fuel_consumption=True,
                               rem_engine_capacity=True)
print(df.info())
df = co2.data.dummify_all_features(df)
print(df.info())


Incomplete rows dropped:251
TOTAL NUMBER OF ROWS DROPPED:251
<class 'pandas.core.frame.DataFrame'>
Index: 4749 entries, 9871943 to 7877195
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Pool                  4749 non-null   object 
 1   MassRunningOrder      4749 non-null   float64
 2   Co2EmissionsWltp      4749 non-null   float64
 3   BaseWheel             4749 non-null   float64
 4   FuelType              4749 non-null   object 
 5   EnginePower           4749 non-null   float64
 6   InnovativeTechnology  4749 non-null   int64  
 7   ElectricRange         620 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 333.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 4749 entries, 9871943 to 7877195
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   MassRunnin

In [4]:
co2.data.save_processed_data(df, classification=True, pickle=True, filepath='../data/processed/')

Data saved to ../data/processed//co2_classification_20240112-161106


In [5]:
df = co2.data.co2_grade_discretization(df)
df['Co2Grade'].value_counts()

Co2Grade
C    1659
B    1015
A     862
D     734
E     365
F      85
G      29
Name: count, dtype: int64

In [6]:
df['Co2EmissionsWltp'].describe()

count    4749.000000
mean      116.114972
std        51.568603
min         0.000000
25%       112.000000
50%       126.000000
75%       141.000000
max       373.000000
Name: Co2EmissionsWltp, dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4749 entries, 9871943 to 7877195
Data columns (total 30 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   MassRunningOrder                 4749 non-null   float64 
 1   Co2EmissionsWltp                 4749 non-null   float64 
 2   BaseWheel                        4749 non-null   float64 
 3   EnginePower                      4749 non-null   float64 
 4   InnovativeTechnology             4749 non-null   int64   
 5   ElecRange_NO_RANGE               4749 non-null   int64   
 6   ElecRange_0to50                  4749 non-null   int64   
 7   ElecRange_50to100                4749 non-null   int64   
 8   ElecRange_100to150               4749 non-null   int64   
 9   ElecRange_150+                   4749 non-null   int64   
 10  Pool_BMW                         4749 non-null   int64   
 11  Pool_FORD                        4749 non-null   int64   
 12  Po

In [8]:
df['Co2Grade'].isna().sum()

0

In [9]:
co2.styles.display_na(df)

/home/mln/miniconda3/envs/co2/lib/python3.9/site-packages/pandas/io/formats/style.py:4052: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Colonne,Valeurs manquantes (%)
0,MassRunningOrder,0.00%
1,Co2EmissionsWltp,0.00%
2,BaseWheel,0.00%
3,EnginePower,0.00%
4,InnovativeTechnology,0.00%
5,ElecRange_NO_RANGE,0.00%
6,ElecRange_0to50,0.00%
7,ElecRange_50to100,0.00%
8,ElecRange_100to150,0.00%
9,ElecRange_150+,0.00%


In [10]:
co2.styles.displayer(df)

,MassRunningOrder,Co2EmissionsWltp,BaseWheel,EnginePower,InnovativeTechnology,ElecRange_NO_RANGE,ElecRange_0to50,ElecRange_50to100,ElecRange_100to150,ElecRange_150+,Pool_BMW,Pool_FORD,Pool_HYUNDAI,Pool_KIA,Pool_MAZDA-SUBARU-SUZUKI-TOYOTA,Pool_MERCEDES-BENZ,Pool_RENAULT-NISSAN-MITSUBISHI,Pool_STELLANTIS,Pool_TESLA,Pool_VW-SAIC,FuelType_DIESEL,FuelType_DIESEL/ELECTRIC,FuelType_E85,FuelType_ELECTRIC,FuelType_LPG,FuelType_NG,FuelType_NG-BIOMETHANE,FuelType_PETROL,FuelType_PETROL/ELECTRIC,Co2Grade
9871943,1615.000,134.000,2829.000,128.000,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,C
4969321,1313.000,120.000,2667.000,81.000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,B
2183213,1475.000,126.000,2612.000,70.000,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,C
17946,1601.000,148.000,2629.000,110.000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,D
5989334,1248.000,0.000,2417.000,61.000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,A


In [11]:
df['InnovativeTechnology'].value_counts()

InnovativeTechnology
0    2441
1    2308
Name: count, dtype: int64

In [12]:
co2.viz.plot_correlation_heatmap(df)

In [13]:
def dummification(df, column):
    dummies = pd.get_dummies(data=df[column], dtype=int, prefix=f"{column}")
    df = df.join(dummies)
    df.drop(columns=df[column], inplace=True)
    return df

dft = pd.get_dummies(data = df['Pool'], dtype=int, prefix='Pool')
df = df.join(dft)
df.drop('Pool', axis = 1, inplace=True)

# Colonne  'FuelType'
dft = pd.get_dummies(data = df['FuelType'], dtype=int, prefix='Fuel')
df = df.join(dft)
df.drop('FuelType', axis = 1, inplace=True)

df = df.join(pd.get_dummies(data=df['CategoryOf'], dtype=int, prefix='Category'))
df.drop('CategoryOf', axis=1, inplace=True)
# Traitement de la colonne 'CategoryOf'  -->  m1 ou m1g
display(df.head())


KeyError: 'Pool'